In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import json
import datetime

In [3]:
input_lang = "FAQ_ENGLISH"

In [4]:
node_id = Title.objects.get(title=input_lang).page.node.id
question_id = list(TreeNode.objects.filter(parent_id=node_id).values_list('id', flat=True))
ans_df = pd.DataFrame(list(TreeNode.objects.filter(parent_id__in=question_id).values('parent_id', 'id')))
question_id

[24]

In [5]:
value_df = ans_df.groupby('parent_id').agg({'id':list}).reset_index()
value_df

,parent_id,id
0,24,"[25, 26]"


In [54]:
data_dict = {}
data_dict[input_lang] = {}

for index,value in value_df.iterrows():
    page_id = Page.objects.get(node_id=value.parent_id)
    question_type = Title.objects.get(page_id=page_id).title
    
    if not question_type in data_dict[input_lang]:
        data_dict[input_lang][question_type] = []
        
    for question_id in value.id:
        page_id2 = Page.objects.get(node_id=question_id)
        plugin_ids = list(Page.objects.filter(id__exact=page_id2.id).values_list('placeholders__cmsplugin', flat=True))                
        question_obj = Title.objects.get(page_id=page_id2)
        question_dict = {
            'question': question_obj.title,
            'answer': question_obj.meta_description,
            'plugin': ''
        }
        
        for plugin_id in plugin_ids:
            if plugin_id != None or plugin_id != '':
                view = CMSPlugin.objects.get(id=plugin_id).
        data_dict[input_lang][question_type].append(question_dict)
data_dict

[10]


{'FAQ_ENGLISH': {'Questions': [{'question': 'Question1',
    'answer': 'Answer1',
    'plugin': ''},
   {'question': 'Question2', 'answer': 'Answer2', 'plugin': ''}]}}

In [55]:
CMSPlugin.objects.all()

<MP_NodeQuerySet [<cms.models.pluginmodel.CMSPlugin id=10 plugin_type='TextPlugin' object at 0x7f9013bcf490>]>

In [28]:
expense_log_list = list(ExpenseLog.objects.filter(crop_id=2).order_by('date').values_list('id', 'expense_cv', 'expense_cv__name', 'cost', 'date', 'crop', 'crop__crop_cv__name', 'crop__area_in_hectare', 'crop__sowing_year__year'))
expense_log_column = ['expense_id', 'expense_cv_id', 'expense_cv_name', 'cost', 'date', 'crop_id', 'crop_name', 'area', 'showing_year']
expense_df = pd.DataFrame(expense_log_list, columns=expense_log_column)
year = datetime.datetime.now().year
expense_df['curent_year'] = year
expense_df['age'] = expense_df['curent_year'] - expense_df['showing_year']
expense_df.to_dict('r')

[{'expense_id': 1,
  'expense_cv_id': 1,
  'expense_cv_name': 'planting',
  'cost': Decimal('300.00'),
  'date': datetime.date(2021, 3, 1),
  'crop_id': 2,
  'crop_name': 'crop one',
  'area': Decimal('23.000'),
  'showing_year': 2009,
  'curent_year': 2021,
  'age': 12}]

In [25]:
year = datetime.datetime.now().year
year

2021

# Stackhokder's Directory

### Farmer

In [6]:
farmer_value_list = list(Farmer.objects.all().values_list('user_profile__user__first_name', 'user_profile__user__last_name', 'user_profile__mobile', 'state', 'district', 'taluk', 'block', 'revenue_village', 'village', 'street', 'pincode', 'latitude', 'longitude'))                                                             
farmer_value_column = ['first_name', 'last_name', 'mobile', 'state', 'district', 'taluk', 'block', 'revenue_village', 'village', 'street', 'pincode', 'latitude', 'longitude']
farmer_df = pd.DataFrame(farmer_value_list, columns=farmer_value_column)
farmer_df

,first_name,last_name,mobile,state,district,taluk,block,revenue_village,village,street,pincode,latitude,longitude
0,vivek,rk,9751369166,1,1,None,4,1,None,None,614402,None,None
1,vivek,rk,9751369166,1,1,None,4,1,None,None,614402,None,None


### Industry

In [80]:
industry_obj = IndustryOffice.objects.filter()
industry_official_obj = IndustryOfficial.objects.all()

industry_value_list = list(industry_obj.values_list('id', 'industry__name', 'village', 'street', 'taluk', 'block__name', 'district__name', 'industryofficecropmap__crop_cv__name'))                                                                                                       
industry_value_column = ['industry_office_id', 'industry_name', 'village', 'street', 'taluk', 'block', 'district', 'crop']
industry_df = pd.DataFrame(industry_value_list, columns=industry_value_column)
industry_df = industry_df.groupby(['industry_office_id', 'industry_name', 'village', 'street', 'taluk', 'block', 'district']).agg({'crop': 'unique'}).reset_index()

official_value_list = list(industry_official_obj.values_list('industry_office_id', 'user_profile__user__first_name', 'user_profile__user__last_name', 'user_profile__mobile', 'is_contact_person'))
official_value_column = ['industry_office_id', 'first_name', 'last_name', 'mobile', 'contact_person']
official_df = pd.DataFrame(official_value_list, columns=official_value_column)
official_df = official_df[official_df['contact_person']==True]

final_df = pd.merge(industry_df, official_df, left_on='industry_office_id', right_on='industry_office_id', how='left')
data_dict = final_df.fillna('').to_dict('r')


/home/sunesh/Software/virtualenv/jamun/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


### Nursery

In [79]:
nursery_office_obj = NurseryOffice.objects.filter()
nursery_incharge_obj = NurseryIncharge.objects.all()

nursery_value_list = list(nursery_office_obj.values_list('id','nursery__name', 'street', 'village', 'taluk', 'pincode', 'block__name', 'district__name', 'state__name', 'is_head_office', 'nurseryofficecropmap__clone__name', 'nurseryofficecropmap__clone__crop_cv__name'))                                                                                                                     
nursery_value_column = ['nursery_office_id', 'nursery_name', 'street', 'village', 'taluk', 'pincode', 'block', 'district', 'state', 'is_head_office', 'clone_name', 'crop_name']
nursery_df = pd.DataFrame(nursery_value_list, columns=nursery_value_column)

nursery_df['crop_clone'] =  nursery_df['crop_name'] + ' - ' + nursery_df['clone_name']
nursery_crop_df = nursery_df.groupby('nursery_office_id').agg({'crop_clone': 'unique'})
nursery_df = nursery_df.drop(columns=['crop_name', 'clone_name'])

nursery_df = nursery_df.groupby(['nursery_office_id', 'nursery_name', 'street', 'village', 'taluk', 'pincode', 'block', 'district', 'state', 'is_head_office',]).agg({'crop_clone': 'unique'}).reset_index()
data_dict = nursery_df.to_dict('r')

incharge_value_list = list(nursery_incharge_obj.values_list('nursery_office_id', 'user_profile__user__first_name', 'user_profile__user__last_name', 'user_profile__mobile', 'is_contact_person'))
incharge_value_column = ['nursery_office_id', 'first_name', 'last_name', 'mobile', 'is_contatct_person']
incharge_df = pd.DataFrame(incharge_value_list, columns=incharge_value_column)
incharge_df = incharge_df[incharge_df['is_contatct_person']==True]

final_df = pd.merge(nursery_df, incharge_df, left_on='nursery_office_id', right_on='nursery_office_id', how='left')
data_dict = final_df.fillna('').to_dict('r')
# data_dict

/home/sunesh/Software/virtualenv/jamun/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


### IFGTB

In [88]:
institute_office_obj = InstituteOffice.objects.all()
scientist_obj = Scientist.objects.all()

institute_office_list = list(institute_office_obj.values_list('id','institute__name', 'street', 'village', 'taluk', 'pincode', 'block__name', 'district__name', 'state__name', 'is_head_office'))                                                                                                              
institute_office_column = ['institute_office_id', 'institute_name', 'street', 'village', 'taluk', 'pincode', 'block', 'district', 'state', 'is_head_office']
institute_office_df = pd.DataFrame(institute_office_list, columns=institute_office_column)
institute_office_df

scientist_value_list = list(scientist_obj.values_list('institute_office_id', 'user_profile__user__first_name', 'user_profile__user__last_name', 'user_profile__mobile', 'is_contact_person', 'expertise__name'))
scientist_value_column = ['institute_office_id', 'first_name', 'last_name', 'mobile', 'contatct_person', 'expertise']
scientist_df = pd.DataFrame(scientist_value_list, columns=scientist_value_column)
scientist_df = scientist_df[scientist_df['contatct_person']==True]

final_df = pd.merge(institute_office_df, scientist_df, left_on='institute_office_id', right_on='institute_office_id', how='left').fillna('')
data_dict = final_df.fillna('').to_dict('r')
final_df

/home/sunesh/Software/virtualenv/jamun/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,institute_office_id,nursery_name,street,village,taluk,pincode,block,district,state,is_head_office,first_name,last_name,mobile,contatct_person,expertise
0,1,IFGTB1,lrg nagar,NSN Palayam,kurudampalayam,641031,Thondamuthur,Coimbatore,TamilNadu,True,industry person1,a,9751369167.0,True,leaf
1,3,IFGTB1,lrd nagar,NSp Palayam,kurudampalayam,641033,Thondamuthur,Coimbatore,TamilNadu,False,industry person5,e,9751369171.0,True,tree branch
2,4,IFGTB2,lru nagar,NSk Palayam,kurudampalayam,641034,Thondamuthur,Coimbatore,TamilNadu,False,,,,,
3,5,IFGTB1,lrl nagar,NSd Palayam,kurudampalayam,641035,Thondamuthur,Coimbatore,TamilNadu,False,,,,,
4,6,IFGTB2,lrk nagar,NSM Palayam,kurudampalayam,641036,Thondamuthur,Coimbatore,TamilNadu,False,,,,,
5,2,IFGTB2,lrk nagar,NSR Palayam,kurudampalayam,641032,Thondamuthur,Coimbatore,TamilNadu,True,industry person3,c,9751369169.0,True,root


In [6]:
User.objects.all().exclude(userprofile__user_type_id=2).values_list('id', 'userprofile__user_type__name')

<QuerySet [(3, 'Farmer'), (8, 'Farmer'), (9, 'Farmer'), (10, 'Farmer'), (16, 'Farmer'), (17, 'Farmer'), (18, 'Farmer'), (20, 'Farmer'), (22, 'Farmer'), (23, 'Farmer'), (25, 'Farmer'), (26, 'Farmer'), (27, 'Farmer'), (28, 'Farmer'), (29, 'Farmer'), (30, 'Farmer'), (31, 'Farmer'), (32, 'Farmer'), (33, 'Farmer'), (1, None), '...(remaining elements truncated)...']>

In [8]:
data_dict = {
            'Farmer': {
                'toal_count':12,
                'totao_acre': 40,
                'district_wise': [
                    {'district': 'district1', 'count': 3, 'acre': 10},
                    {'district': 'district2', 'count': 3, 'acre': 10},
                    {'district': 'district3', 'count': 3, 'acre': 10},
                    {'district': 'district4', 'count': 3, 'acre': 10}
                ]
            },
            'Scientist': {
                'toal_count':5,
                'district_wise': [
                    {'district': 'district1', 'count': 1},
                    {'district': 'district2', 'count': 1},
                    {'district': 'district3', 'count': 1},
                    {'district': 'district4', 'count': 2}
                ]

            },
            'Industry':{
                'toal_count':5,
                'district_wise': [
                    {'district': 'district1', 'count': 1},
                    {'district': 'district2', 'count': 1},
                    {'district': 'district3', 'count': 1},
                    {'district': 'district4', 'count': 2}
                ]
            }
        }

In [9]:
data_dict

{'Farmer': {'toal_count': 12,
  'totao_acre': 40,
  'district_wise': [{'district': 'district1', 'count': 3, 'acre': 10},
   {'district': 'district2', 'count': 3, 'acre': 10},
   {'district': 'district3', 'count': 3, 'acre': 10},
   {'district': 'district4', 'count': 3, 'acre': 10}]},
 'Scientist': {'toal_count': 5,
  'district_wise': [{'district': 'district1', 'count': 1},
   {'district': 'district2', 'count': 1},
   {'district': 'district3', 'count': 1},
   {'district': 'district4', 'count': 2}]},
 'Industry': {'toal_count': 5,
  'district_wise': [{'district': 'district1', 'count': 1},
   {'district': 'district2', 'count': 1},
   {'district': 'district3', 'count': 1},
   {'district': 'district4', 'count': 2}]}}

In [ ]:
# import requests
# try:
#     from StringIO import StringIO
# except ImportError:
    
#     from io import StringIO
#     from io import BytesIO
    
# from decimal import Decimal
# google_sheet_url = "https://docs.google.com/spreadsheets/d/1hdYgyHUqCesUfr22PpYx_VgA88A2QavcKSRP011sCRY/export?format=xlsx"
# r = requests.get(google_sheet_url)
# data = r.content
# df = pd.read_excel(BytesIO(data),sheet_name='farmer_geo_fence')
# df = df.fillna(0)

###  Farmer Geo Fence

ModuleNotFoundError: No module named 'hashers'

### 

In [14]:
sheet_id = "1hdYgyHUqCesUfr22PpYx_VgA88A2QavcKSRP011sCRY"
sheet_name = "farmer_geo_fence"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_from_excel = pd.read_csv(url)

# df_from_excel = df_from_excel.dropna(how='all', axis='columns')
excel_columns = df_from_excel.columns

# rename_columns
for column_name in excel_columns:
    new_column_name = column_name.lower()
    if " " in new_column_name:
        new_column_name = new_column_name.replace(' ', '_')
        if new_column_name[-1] == "_":
            new_column_name = new_column_name[:-1]

    df_from_excel = df_from_excel.rename(columns={column_name: new_column_name})
df_from_excel
df_from_excel = df_from_excel.fillna(0)

#------------------data_upload_for_farmer_profile--------------------------#
already_exists = 0
uploaded_count = 0
for index, value in df_from_excel.iterrows():
    if User.objects.filter(username=value['mobile']).exists():
        already_exists += 1
    else:
        uploaded_count += 1
        user = User.objects.create(
            first_name=value['first_name'],
            last_name=value['last_name'],
            username=value['mobile'],
            password=make_password('1234')
        )
        print('user saved')
        user_profile = UserProfile.objects.create(
            user=user,
            user_type_id=1,
            mobile=value['mobile'],
            language_preference_id=1
        )
        value_dict = value.to_dict()
        if value.gender == 'Male':
            value_dict['gender_id'] = 1
        elif value.gender == 'Female':
            value_dict['gender_id'] = 2
        else:
            value_dict['gender_id'] = 3

        status_code, data = post_data_to_micro_service(1, '/v1/main/save/farmer/for/geo/fence/', value_dict )
        user_profile.ms_farmer_code = data['farmer_code']
        print("this is access code", data)
        user_profile.save()

        data['token'] = make_token(value['mobile'], '1234')
        data['user_type_id'] = user_profile.user_type_id
        data['user_profile_id'] = user_profile.id

# transaction.savepoint_commit(sid)

NameError: name 'make_password' is not defined

In [17]:
pd.DataFrame(list(Scientist.objects.all().values('id', 'designation', 'expertise')))

,id,designation,expertise
0,7,test Scientist,1
1,7,test Scientist,2


In [15]:
list(Scientist.objects.filter(institute_office_id=3, is_contact_person=True).values_list('expertise__'))

[(1,), (2,)]

In [12]:
user_query = UserQuery.objects.all().order_by('-requested_date')
user_query_ids = user_query.values_list('id', flat=True)
user_query_list = list(user_query.values_list('id', 'user__id', 'query_type__id', 'query_type__name',
                                              'is_assigned', 'requested_date', 'notes', 'status', 'status__name', 'voice', 'time_created','area_in_acre', 'age_in_year', 'age_in_month', 'clone__name', 'user__first_name', 'title', 'user__main__userprofile__ms_farmer_code')) #user__userprofile__ms_farmer_code
user_query_column = ['user_query_id', 'user_id', 'query_type_id', 'query_type', 'is_assigned',
                     'requested_date', 'notes', 'status_id', 'status_name', 'voice', 'time_created', 'area_in_acre', 'age_in_year', 'age_in_month', 'crop_name', 'first_name', 'title', 'user_profile']
user_query_df = pd.DataFrame(user_query_list, columns=user_query_column)

FieldError: Cannot resolve keyword 'main' into field. Choices are: auth_token, checkbox_answer_added_by, created_usergroups, created_users, crop, cropgeofence, cropgeotag, date_joined, djangocms_usersettings, dropdown_answer_added_by, email, estimated_by, farmer, farmer_created_by, farmer_modified_by, farmer_user, farmerbulkuploadlog, fcmdevice, filer_clipboards, filer_folder_permissions, filer_owned_folders, first_name, globalpagepermission, groups, id, is_active, is_staff, is_superuser, landgeofence, landgeotag, last_login, last_name, logentry, new_user, number_answer_added_by, owned_files, pagepermission, pageuser, password, password_answer_added_by, posted_by, query_assigned_by_user, query_assigned_to_user, querycategoryexpertusermap, radio_answer_added_by, recommended_by_expert_user, super_admin_reviewed, superior_user, surveyorofficialmap, text_answer_added_by, user_created, user_modified, user_permissions, username, userquery, waterresource, yieldformula

In [24]:
User.objects.filter(id=1).values_list('userprofile__id')

FieldError: Cannot resolve keyword 'userprofile' into field. Choices are: auth_token, checkbox_answer_added_by, created_usergroups, created_users, crop, cropgeofence, cropgeotag, date_joined, djangocms_usersettings, dropdown_answer_added_by, email, estimated_by, farmer, farmer_created_by, farmer_modified_by, farmer_user, farmerbulkuploadlog, fcmdevice, filer_clipboards, filer_folder_permissions, filer_owned_folders, first_name, globalpagepermission, groups, id, is_active, is_staff, is_superuser, landgeofence, landgeotag, last_login, last_name, logentry, new_user, number_answer_added_by, owned_files, pagepermission, pageuser, password, password_answer_added_by, posted_by, query_assigned_by_user, query_assigned_to_user, querycategoryexpertusermap, radio_answer_added_by, recommended_by_expert_user, super_admin_reviewed, superior_user, surveyorofficialmap, text_answer_added_by, user_created, user_modified, user_permissions, username, userquery, waterresource, yieldformula

<QuerySet [2, None, None, None, None, None, None, None, None, None, 3, None, None, None, None, None, None, None, None, None, '...(remaining elements truncated)...']>

In [14]:
notification_list = NotificatiionLog.objects.filter(user_id=1630, is_active=True).order_by('-id').values_list('id', 'recommendation__user_query_id', 'recommendation__user_query__requested_date', 'recommendation__user_query__notes', 'recommendation__user_query__query_type__id', 'recommendation__user_query__query_type__name',
                            'recommendation__user_query__status', 'recommendation__user_query__status__name', 'recommendation__user_query__time_created', 'recommendation__user_query__area_in_acre', 'recommendation__user_query__age_in_year', 'recommendation__user_query__age_in_month', 'recommendation__user_query__title', 'recommendation__user_query__clone__name', 'recommendation__user_query__clone__crop_cv__name', 'recommendation__user_query__voice', 
                            'notification_type__name', 'notification_type_id', 'notification_type__color', 'notification_type__background', 'is_sceen', 'notification_text', 'time_created')
notification_column = ['id', 'user_query_id', 'requested_date', 'notes', 'query_type', 'query_type__name',
                    'status_id', 'status_name', 'time_created', 'area_in_acre', 'age_in_year', 'age_in_month', 'title', 'clone_name', 'crop_name', 'voice_path', 'notification_type', 'notification_type_id', 'color', 'background', 'is_seen', 'notification_text', 'time_created']
notification_df = pd.DataFrame(notification_list, columns=notification_column)
# if not notification_df.empty:
#     notification_df['audio'] = notification_df.apply(lambda x: convert_wav_to_base64(x['voice_path']), axis=1)
#     notification_df['play_audio'] = False
#     notification_df = notification_df.fillna('')
notification_df= notification_df.sort_values(by = ['is_seen','id'], ascending = False)
notification_df

,id,user_query_id,requested_date,notes,query_type,query_type__name,status_id,status_name,time_created,area_in_acre,...,clone_name,crop_name,voice_path,notification_type,notification_type_id,color,background,is_seen,notification_text,time_created
0,96,81.0,2022-01-09,sunesh is trying to handle everything himself,1.0,Scientist,2.0,Recommented,2022-01-09 16:25:14.777783,1.000,...,EC-9,Eucalyptus,diagnosis/user_query/2022-01-09/user_1630/quer...,Recommendation,2,#282345,#ebf1fa87,True,sss,2022-01-09 23:55:55.221715
1,95,81.0,2022-01-09,sunesh is trying to handle everything himself,1.0,Scientist,2.0,Recommented,2022-01-09 16:25:14.777783,1.000,...,EC-9,Eucalyptus,diagnosis/user_query/2022-01-09/user_1630/quer...,Recommendation,2,#282345,#ebf1fa87,True,sunsh,2022-01-09 23:46:30.539040
2,91,81.0,2022-01-09,sunesh is trying to handle everything himself,1.0,Scientist,2.0,Recommented,2022-01-09 16:25:14.777783,1.000,...,EC-9,Eucalyptus,diagnosis/user_query/2022-01-09/user_1630/quer...,User Query,1,#282345,#f9eaea87,True,ok ok this is kunalan 2,2022-01-09 21:04:14.157365
3,90,81.0,2022-01-09,sunesh is trying to handle everything himself,1.0,Scientist,2.0,Recommented,2022-01-09 16:25:14.777783,1.000,...,EC-9,Eucalyptus,diagnosis/user_query/2022-01-09/user_1630/quer...,Recommendation,2,#282345,#ebf1fa87,True,this is kunalan,2022-01-09 21:01:00.299863
4,87,81.0,2022-01-09,sunesh is trying to handle everything himself,1.0,Scientist,2.0,Recommented,2022-01-09 16:25:14.777783,1.000,...,EC-9,Eucalyptus,diagnosis/user_query/2022-01-09/user_1630/quer...,Recommendation,2,#282345,#ebf1fa87,True,this is not gud,2022-01-09 19:34:40.315285
5,84,NaN,None,None,NaN,None,NaN,None,NaT,None,...,None,None,None,General,3,#282345,#faf2eb91,True,sdgf,2022-01-09 10:27:33.998869
6,81,NaN,None,None,NaN,None,NaN,None,NaT,None,...,None,None,None,General,3,#282345,#faf2eb91,True,fsfswsaasdsfdwe,2022-01-09 10:26:18.409675
7,78,NaN,None,None,NaN,None,NaN,None,NaT,None,...,None,None,None,General,3,#282345,#faf2eb91,True,sssdaf,2022-01-09 10:25:45.759599
8,75,NaN,None,None,NaN,None,NaN,None,NaT,None,...,None,None,None,General,3,#282345,#faf2eb91,True,ss,2022-01-09 10:25:23.210513
9,72,NaN,None,None,NaN,None,NaN,None,NaT,None,...,None,None,None,General,3,#282345,#faf2eb91,True,can i use this,2022-01-09 10:21:13.706621


In [15]:
for i in NotificatiionLog.objects.all():
    i.is_sceen = False
    i.save()